Stap 1:
Haal 50.000 gebruikers uit de csv file

In [14]:
userdata = sc.textFile("C:/Users/Troley/Dropbox/HBO-ICT/Blok 7 2k17/Big Data/train20m.csv")
sp = userdata.map(lambda x: tuple(x.split('::')))
bla = sp.map(lambda x: [x[0]])
print(bla.take(10))

[['11973'], ['11973'], ['11973'], ['11973'], ['11973'], ['11973'], ['11973'], ['11973'], ['11973'], ['11973']]


Stap 2:
Distinct films eruit halen

In [38]:
filmsIDOnly = splittedUserDatardd.map(lambda x: x[1]).distinct()
print(filmsIDOnly.take(10))

['1', '10', '100', '1000', '100003', '100006', '100008', '100010', '100013', '100015']
